In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import os
import torch
import torchvision
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import albumentations

In [ ]:
import torch
import torch.nn as nn
from skimage import io, transform

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,image_path,targets,augmentations=None):
        self.image_path = image_path
#         self.features = features
        self.targets = targets
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,item):
        image = io.imread(self.image_path[item])
#         features = self.features[item,:]
        targets = self.targets[item]
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
            
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        return {
            "image": torch.tensor(image, dtype=torch.float),
#             "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

In [ ]:
import torch
import torch.nn as nn

model_name = 'tf_efficientnet_b2_ns'
# file_name = "../input/tf-efficientnet-b2-ns/tf_efficientnet_b2_ns-00306e48.pth"
out_dim    = 1

def get_model():
    model = timm.create_model(model_name, pretrained=False)
    
#     model.load_state_dict(torch.load(file_name))
    model.classifier = nn.Linear(model.classifier.in_features, 
                             out_dim)
    return model

In [ ]:
import os
from sklearn import metrics
import albumentations
device = 'cuda'
epochs = 12
data_path = '../input/petfinder-pawpularity-score'

valid_aug = albumentations.Compose(
    [
        albumentations.Resize(256, 256, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)


In [ ]:
from itertools import chain
device = 'cuda'
model_f = get_model()
model_f.to(device)
model_f.load_state_dict(torch.load('../input/model-0-effnet/model-epoch0.pth'))
data_path = '../input/petfinder-pawpularity-score'
device = 'cuda'
df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_images = df_test.Id.values.tolist()
test_images = [os.path.join(data_path,'test',i + '.jpg') for i in test_images]

test_dataset =  CustomDataset(image_path = test_images,targets = np.ones(len(test_images)),augmentations=valid_aug)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64,shuffle=False) 


final_outputs = []
 
with torch.no_grad():
    for data in test_loader:
        inputs = data['image']
        inputs = inputs.to(device, dtype=torch.float)
        output = model_f(inputs)
        output = output.detach().cpu().numpy().tolist()
        final_outputs.extend(output)
        

final_outputs = list(chain.from_iterable(final_outputs))        
submission = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submission['Pawpularity'] = final_outputs
submission.to_csv('submission.csv',index = False)